In [ ]:
import visa
import rigol1000z as Rigol1000z
from datetime import datetime
import time
from IPython.display import Image

In [ ]:
rm = visa.ResourceManager()

def screenshot(file_suff):
    rm = visa.ResourceManager()
    res = rm.list_resources()
    #print( res )
    ins = rm.get_instrument( res[0] ) # !

    ins.write(":DISP:DATA?")
    bmpdata = ins.read_raw()[2+9:]

    ins.close()

    # save image file
    name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    print("Enter a filename-safe description (or none). Hit return.")
    comment = file_suff
    if( len(comment) ):
        name = name + "_" + comment

    try:
        from PIL import Image

    except ImportError as e:
        print("PIL(low) not imported because:", e)
        filename = name + ".png"
        print("Saving screen as", filename)
        with open( filename, "wb") as f:
            f.write( bmpdata )
    return filename

In [ ]:
osc_resource = rm.open_resource(rm.list_resources()[0])

osc = Rigol1000z.Rigol1000z(osc_resource)

osc.timebase.set_timebase_scale_s_div(100E-3)
for i in range(1,5):
    osc[i].set_vertical_scale_V(1)
    osc[i].set_probe_ratio(1)
    osc[i].visa_write(':disp 1')
    osc.visa_write(":chan%d:OFFSet %.2f"%(i,(i*2)-5))

# Stop the scope.
osc.stop()
osc.set_single_shot()

while("STOP" not in str(osc.visa_ask(":TRIG:STAT?"))):
    print(".")
    time.sleep(0.5)
    
osc.visa_resource.close()
filename = screenshot("SaveTest23")
print("Filename " + filename)
Image(filename=filename) 